In [1]:
import pandas as pd
import numpy as np
import time
import datetime 
from dateutil.parser import parse
from datetime import date, timedelta
from pandas.tseries.offsets import *

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import *
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn import *
from catboost import CatBoostRegressor

from lightgbm import LGBMRegressor
import pickle
from lunar import *
from copy import deepcopy

%matplotlib inline

/home/yuanhao/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/yuanhao/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/yuanhao/anaconda3/lib/python3.6/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWar

In [2]:
data = pd.read_csv('holiday_v2.csv')

In [3]:
data.head()

,virtual_date,hld
0,2012-12-31,0
1,2013-01-01,1
2,2013-01-02,1
3,2013-01-03,1
4,2013-01-04,0


In [4]:
data.tail()

,virtual_date,hld
1822,2017-12-27,0
1823,2017-12-28,0
1824,2017-12-29,0
1825,2017-12-30,1
1826,2017-12-31,1


In [5]:
data['virtual_date'] = pd.to_datetime(data['virtual_date'])

In [6]:
data['dow'] = data['virtual_date'].dt.dayofweek

In [7]:
data['month'] = data['virtual_date'].dt.month
data['day'] = data['virtual_date'].dt.day

In [8]:
data = data.reset_index()

In [9]:
holiday_df = pd.read_csv('holiday_v2.csv')
holiday_df['virtual_date'] = pd.to_datetime(holiday_df['virtual_date'])
holiday_series = holiday_df.set_index('virtual_date')

data.loc[1:,'yesterday_hld'] = data.loc[1:,'virtual_date'].apply(lambda x: holiday_series.loc[x+DateOffset(days=-1),'hld'])
data.loc[0:1825,'tomorrow_hld'] = data.loc[0:1825,'virtual_date'].apply(lambda x: holiday_series.loc[x+DateOffset(days=1),'hld'])

In [10]:
data['ith_workday']=np.nan
data['ith_workday_r'] = np.nan
for idx, x in data.iterrows():
    if idx==0:
        data.loc[idx, 'ith_workday'] = 0
        continue
    elif x['hld']==1:
        data.loc[idx, 'ith_workday'] = -1
        data.loc[idx, 'ith_workday_r'] = -1
        idx_temp, counter = idx-1, 1
        while True:
            if idx_temp==0 or data.loc[idx_temp, 'hld']==1:
                break
            data.loc[idx_temp, 'ith_workday_r'] = counter
            counter += 1
            idx_temp -= 1
    elif x['hld']==0 and x['yesterday_hld'] == 1:
        data.loc[idx, 'ith_workday'] = 1
    else:
        data.loc[idx, 'ith_workday'] = data.loc[idx-1, 'ith_workday'] + 1
    

In [11]:
data.head(15)

,index,virtual_date,hld,dow,month,day,yesterday_hld,tomorrow_hld,ith_workday,ith_workday_r
0,0,2012-12-31,0,0,12,31,NaN,1.0,0.0,NaN
1,1,2013-01-01,1,1,1,1,0.0,1.0,-1.0,-1.0
2,2,2013-01-02,1,2,1,2,1.0,1.0,-1.0,-1.0
3,3,2013-01-03,1,3,1,3,1.0,0.0,-1.0,-1.0
4,4,2013-01-04,0,4,1,4,1.0,0.0,1.0,8.0
5,5,2013-01-05,0,5,1,5,0.0,0.0,2.0,7.0
6,6,2013-01-06,0,6,1,6,0.0,0.0,3.0,6.0
7,7,2013-01-07,0,0,1,7,0.0,0.0,4.0,5.0
8,8,2013-01-08,0,1,1,8,0.0,0.0,5.0,4.0
9,9,2013-01-09,0,2,1,9,0.0,0.0,6.0,3.0


In [16]:
def most_similar_date(x):
    d = x['virtual_date']
    temp = d+DateOffset(years=-1)
    if d.date()>=date(2016,10,12):
        temp = d+DateOffset(years=-2)
    for r in range(5):
        t2 = data[data['virtual_date']==temp + DateOffset(days=r)].reset_index()
        if len(t2)>0:
            if t2.loc[0, 'ith_workday']==x['ith_workday']:
                return temp + DateOffset(days=r)
        t2 = data[data['virtual_date']==temp + DateOffset(days=-r)].reset_index()
        if len(t2)>0 and t2.loc[0,'ith_workday']==x['ith_workday']:
            return temp + DateOffset(days=r)
    return temp

In [17]:
data['s_date'] = data.apply(lambda x:most_similar_date(x), axis=1)

In [18]:
def get_spring_distance(x):
    d = x['virtual_date']
    y = x['virtual_date'].year
    dates={2012:date(2012,1,23),
           2013:date(2013,2,10),
           2014:date(2014,1,31),
           2015:date(2015,2,19),
           2016:date(2016,2,8),
           2017:date(2017,1,28)}
    return (d.date()-dates[y]).days

def get_ld_distance(x):
    d = x['virtual_date']
    y = x['virtual_date'].year
    return (d.date()-date(y,5,1)).days

def get_nd_distance(x):
    d = x['virtual_date']
    y = x['virtual_date'].year
    return (d.date()-date(y,10,1)).days

def get_ny_distance(x):
    d = x['virtual_date']
    y = x['virtual_date'].year
    return (d.date()-date(y,1,1)).days

def get_ny_distance2(x):
    d = x['virtual_date']
    y = x['virtual_date'].year
    return (d.date()-date(y+1,1,1)).days

In [19]:
data['spring_distance'] = data.apply(lambda x: get_spring_distance(x), axis=1)
data['ld_distance'] = data.apply(lambda x: get_ld_distance(x), axis=1)
data['nd_distance'] = data.apply(lambda x: get_nd_distance(x), axis=1)
data['ny_distance'] = data.apply(lambda x: get_ny_distance(x), axis=1)
data['ny_distance2'] = data.apply(lambda x: get_ny_distance2(x), axis=1)

In [20]:
data[['virtual_date','ith_workday','s_date','ith_workday_r','spring_distance',\
     'ld_distance','nd_distance','ny_distance','ny_distance2']].to_csv('data_feature-v2.csv',index=False)